In [147]:
import requests
import pandas as pd
import geopandas as gpd
import numpy as np
import folium

In [148]:
url_casillas="https://datamx.io/dataset/985fceef-1d7b-49ed-8098-a5bc182462d2/resource/50f1cc66-c303-4713-adc4-dcd27bd4c3c3/download/casillas.gpkg"
casillas=gpd.read_file(url_casillas)
#filtrar entidad 1
casillas=casillas[casillas["entidad"]==1]
casillas

,id,domicilio,ubicacion,referencia,name,alias,version,entidad,municipio,municipio_name,localidad,localidad_name,distrito_federal,distrito_local,seccion,manzana,geometry
0,000103380100,"AVENIDA MIGUEL HIDALGO, NÚMERO 18,COLONIA CEN...","ESCUELA PRIMARIA IGNACIO ZARAGOZA,TURNO MATUTINO",ENTRE CALLE MELCHOR OCAMPO Y CALLE DE LA JUVENTUD,B,Básica,2024,1,2,ASIENTOS,1,ASIENTOS,1,2,338,19,POINT (-102.08824 22.23659)
1,010103380200,"AVENIDA MIGUEL HIDALGO, NÚMERO 18,COLONIA CEN...","ESCUELA PRIMARIA IGNACIO ZARAGOZA,TURNO MATUTINO",ENTRE CALLE MELCHOR OCAMPO Y CALLE DE LA JUVENTUD,C2,Contigua 2,2024,1,2,ASIENTOS,1,ASIENTOS,1,2,338,19,POINT (-102.08824 22.23659)
2,010103380100,"AVENIDA MIGUEL HIDALGO, NÚMERO 18,COLONIA CEN...","ESCUELA PRIMARIA IGNACIO ZARAGOZA,TURNO MATUTINO",ENTRE CALLE MELCHOR OCAMPO Y CALLE DE LA JUVENTUD,C1,Contigua 1,2024,1,2,ASIENTOS,1,ASIENTOS,1,2,338,19,POINT (-102.08824 22.23659)
3,000103390100,"CALLE MATAMOROS, SIN NÚMERO, BARRIO DEL CECYTE...",COLEGIO DE ESTUDIOS CIENTÍFICOS Y TECNOLÓGICOS...,ENTRE CALLES DEL PARQUE Y ANÁHUAC,B,Básica,2024,1,2,ASIENTOS,1,ASIENTOS,1,2,339,45,POINT (-102.08518 22.24611)
4,010103390200,"CALLE MATAMOROS, SIN NÚMERO, BARRIO DEL CECYTE...",COLEGIO DE ESTUDIOS CIENTÍFICOS Y TECNOLÓGICOS...,ENTRE CALLES DEL PARQUE Y ANÁHUAC,C2,Contigua 2,2024,1,2,ASIENTOS,1,ASIENTOS,1,2,339,45,POINT (-102.08518 22.24611)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,010106820300,"AVENIDA PASEO SAN GERARDO, NÚMERO 207, FRACCIO...",UNIVERSIDAD POLITÉCNICA DE AGUASCALIENTES,ENTRE AVENIDA MAHATMA GANDHI Y AVENIDA PASEO D...,C3,Contigua 3,2024,1,1,AGUASCALIENTES,328,FRACC. SAN GERARDO,3,9,682,15,POINT (-102.29732 21.80567)
1682,010106820400,"AVENIDA PASEO SAN GERARDO, NÚMERO 207, FRACCIO...",UNIVERSIDAD POLITÉCNICA DE AGUASCALIENTES,ENTRE AVENIDA MAHATMA GANDHI Y AVENIDA PASEO D...,C4,Contigua 4,2024,1,1,AGUASCALIENTES,328,FRACC. SAN GERARDO,3,9,682,15,POINT (-102.29732 21.80567)
1683,000106830100,"CALLE 20 DE NOVIEMBRE, SIN NÚMERO, LOCALIDAD M...",ESCUELA PRIMARIA CRISTÓBAL COLÓN,ENTRE CALLE CRISTÓBAL COLÓN Y CALLE LEYES DE R...,B,Básica,2024,1,1,AGUASCALIENTES,218,MONTORO,3,9,683,8,POINT (-102.30271 21.75497)
1684,010106830100,"CALLE 20 DE NOVIEMBRE, SIN NÚMERO, LOCALIDAD M...",ESCUELA PRIMARIA CRISTÓBAL COLÓN,ENTRE CALLE CRISTÓBAL COLÓN Y CALLE LEYES DE R...,C1,Contigua 1,2024,1,1,AGUASCALIENTES,218,MONTORO,3,9,683,8,POINT (-102.30271 21.75497)


In [150]:
casillas["distrito_federal"].unique()

array(['1', '2', '3'], dtype=object)

In [151]:
#Url básica
base_url = "https://prep2024.ine.mx/publicacion/nacional/assets/presidencia/entidad/1_"

# Listas para almacenar los datos
tipo_casilla = []
votos = []
secciones = []
aliases = []
partidos = []
distritos = [] 

# Lista de partidos y coaliciones
party_list = ["PAN","PRI","PRD","PVEM","PT","MC","MORENA","PAN-PRI-PRD","PAN-PRI","PAN-PRD","PRI-PRD","PVEM-PT-MORENA","PVEM-PT","PVEM-MORENA","PT-MORENA"]


In [152]:
# Iterar sobre los 3 distritos federales
for i in range(1, 4):
    url = f"{base_url}{i}.json"
    response = requests.get(url)
    data = response.json()
    
    # Extraer datos de la API
    for item in data["detalleCasillas"]["votosCasillas"]["body"]:
        tipo_casilla.append(item["tipoCasilla"])
        votos.append(item["votosPartidos"])
        seccion = item["tipoCasilla"].split(' ')[0]
        alias = ' '.join(item["tipoCasilla"].split(' ')[1:])
        secciones.append(seccion)
        aliases.append(alias)
        distritos.append(i)

# Dataframe
df = pd.DataFrame({
    'tipo_casilla': tipo_casilla,
    'votos': votos,
    'seccion': secciones,
    'alias': aliases,
    'distrito_federal': distritos
})

#Función para emparejar votos con partidos
def pair_votes_with_parties(vote_list):
    return [(vote, party_list[i]) for i, vote in enumerate(vote_list) if i < len(party_list)]

# Aplicar la función
df['vote_party_pairs'] = df['votos'].apply(pair_votes_with_parties)

df = df.explode('vote_party_pairs')

# Split
df['votos'], df['partido'] = zip(*df['vote_party_pairs'])
df.drop(columns=['vote_party_pairs'], inplace=True)

# Eliminar "Voto Anticipado"
df = df[~df["tipo_casilla"].str.contains("Voto Anticipado")]

# Votos a numérico
df['votos'] = pd.to_numeric(df['votos'], errors='coerce')
df['distrito_federal']=df['distrito_federal'].astype(str)

# Candidatos por partido o coalición
conditions = [
    df['partido'].isin(["PAN", "PRI", "PRD", "PAN-PRI-PRD", "PAN-PRI", "PAN-PRD", "PRI-PRD"]),
    df['partido'] == "MC",
    df['partido'].isin(["PVEM", "PT", "MORENA", "PVEM-PT-MORENA", "PVEM-PT", "PVEM-MORENA", "PT-MORENA"])
]
choices = ['Xóchitl Gálvez', 'Jorge Álvarez Máynez', 'Claudia Sheinbaum']
df['candidato'] = np.select(conditions, choices, default='Other')

In [153]:
#Agrupar por candidato, seccion y alias y sumar votos
df=df.groupby(["distrito_federal","seccion","alias","candidato"]).sum(["votos"]).reset_index()
#Obtener máximo de votos por candidato, sección y alias
df=df.loc[df.groupby(["distrito_federal","seccion","alias"])["votos"].idxmax()]
#Si son ceros eliminar
df=df[df["votos"]!=0]
df

,distrito_federal,seccion,alias,candidato,votos
0,1,338,Básica,Claudia Sheinbaum,239.0
3,1,338,Contigua 1,Claudia Sheinbaum,231.0
9,1,339,Básica,Claudia Sheinbaum,225.0
12,1,339,Contigua 1,Claudia Sheinbaum,219.0
15,1,339,Contigua 2,Claudia Sheinbaum,221.0
...,...,...,...,...,...
5513,3,79,Contigua 1,Xóchitl Gálvez,163.0
5516,3,8,Básica,Xóchitl Gálvez,301.0
5519,3,8,Contigua 1,Xóchitl Gálvez,301.0
5522,3,9,Básica,Xóchitl Gálvez,295.0


In [154]:
casillas_votos=casillas.merge(df,on=["distrito_federal","seccion","alias"])

In [176]:
# Crear un mapa usando una URL de tiles de Google Maps para el estilo 'hybrid'
m = folium.Map(location=[21.88525620,-102.29156770  ], zoom_start=11, tiles='http://mt1.google.com/vt/lyrs=y&z={z}&x={x}&y={y}', attr='Elaborado por el Maestro Karin con información del INE. Programa de Resultados Preliminares 2024 y DataMX')

# Añadir CircleMarkers según el candidato ganador con colores específicos
for i in range(len(casillas_votos)):
    # Determinar el color basado en el candidato ganador
    if casillas_votos.iloc[i]["candidato"] == "Claudia Sheinbaum":
        color = "red"
    elif casillas_votos.iloc[i]["candidato"] == "Xóchitl Gálvez":
        color = "blue"
    else:  # Jorge Álvarez Máynez
        color = "orange"
    
    # Añadir CircleMarker al mapa
    folium.CircleMarker(
        location=[casillas_votos.iloc[i]["geometry"].y, casillas_votos.iloc[i]["geometry"].x],
        radius=5,  # Define el radio del círculo
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        tooltip=folium.Tooltip(text=f"<b>Distrito:</b>{casillas_votos.iloc[i]['distrito_federal']}<br><b>Sección:</b> {casillas_votos.iloc[i]['seccion']}<br><b>Tipo de casilla:</b> {casillas_votos.iloc[i]['alias']}<br><b>Candidato:</b> {casillas_votos.iloc[i]['candidato']}<br><b>Votos:</b> {casillas_votos.iloc[i]['votos']}")
    ).add_to(m)
    
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 200px; height: 90px; 
     border:2px solid grey; z-index:9999;background-color: white; font-size:14px;
     ">&nbsp; <b>Candidaturas</b> <br>
     &nbsp; Claudia Sheinbaum &nbsp; <i class="fa fa-circle" style="color:red"></i><br>
     &nbsp; Xóchitl Gálvez &nbsp; <i class="fa fa-circle" style="color:blue"></i><br>
     &nbsp; Jorge Álvarez Máynez &nbsp; <i class="fa fa-circle" style="color:orange"></i>
</div>
'''

# Añadir leyenda
m.get_root().html.add_child(folium.Element(legend_html))

# Salvar mapa
m.save("mapa_casillas_ags.html")
#Mostrar mapa
m
